<font size="6"><b>S16 T02: Tasca de web scraping</b></font>

# Enunciats
**Nivell 1**
1. Realitza web scraping d'una pàgina de la borsa de Madrid (https://www.bolsamadrid.es) utilitzant BeautifulSoup i Selenium.


**Nivell 2**  

2. Documenta en un word el teu conjunt de dades generat amb la informació que tenen els diferents arxius de Kaggle.


**Nivell 3**  

3. Tria una página web que tu vulguis i realitza web scraping mitjançant la llibreria Scrapy.

## Web scraping la Borsa de Madrid

### amb BeautifulSoup.

In [24]:
from urllib.request import urlopen
from bs4 import BeautifulSoup

url = 'https://www.bolsamadrid.es/esp/aspx/Portada/Portada.aspx'
page = urlopen(url)
soup = BeautifulSoup(page,"html.parser")
print(soup.prettify())

<!DOCTYPE html PUBLIC "-//W3C//DTD XHTML 1.0 Transitional//EN" "http://www.w3.org/TR/xhtml1/DTD/xhtml1-transitional.dtd">
<html xmlns="http://www.w3.org/1999/xhtml">
 <head data-analytics-id="UA-35966870-2" data-app-path="/" data-bolsa="BMadrid" data-hora-act="Thu, 02 Sep 2021 12:11:56 GMT" data-idioma="esp">
  <meta content="IE=11" http-equiv="X-UA-Compatible"/>
  <meta content="text/html; charset=utf-8" http-equiv="Content-Type"/>
  <meta content="Copyright © BME 2021" id="ctl00_copyright" name="copyright"/>
  <title>
   Bolsa de Madrid
  </title>
  <link href="/esp/aspx/RSS/RSS.ashx?feed=Todo" id="ctl00_RSSLink1" rel="alternate" title="Bolsa de Madrid: Todos los contenidos agregados" type="application/rss+xml"/>
  <link href="/esp/aspx/RSS/RSS.ashx?feed=NotasPrensa" id="ctl00_RSSLink2" rel="alternate" title="Bolsa de Madrid: Notas de Prensa" type="application/rss+xml"/>
  <link href="/esp/aspx/RSS/RSS.ashx?feed=Regulacion" id="ctl00_RSSLink3" rel="alternate" title="Bolsa de Madrid: 

In [3]:
for link in soup.find_all('a'):
    if 'href' in link.attrs:
        print(link.attrs['href'])

/?id=ing
/esp/BMadrid/Contacto.aspx
/esp/Inversores/Agenda/HorarioMercado.aspx
/esp/aspx/Inversores/Agenda/Calendario.aspx
/esp/RSS.aspx
/?id=esp
https://www.bolsasymercados.es/
javascript:document.forms.formBusq.submitbusq();
/?id=esp
#
/esp/BMadrid/BMadrid.aspx
/docs/BMadrid/docsSubidos/Historia_Bolsa_ Española.pdf
#
/125_Palacio_Bolsa/
/esp/BMadrid/Palacio/Visita.aspx
/esp/BMadrid/Palacio/VisitaVirtual.aspx
/esp/BMadrid/Palacio/Eventos.aspx
/esp/aspx/Empresas/Eventos/Eventos.aspx?CodBolsa=BMadrid
#
/esp/aspx/Inversores/Formacion/Formacion.aspx?tipo=T&CodBolsa=BMadrid
#
/esp/BMadrid/Protector/Protector1.aspx
/esp/BMadrid/Protector/Protector21.aspx
/esp/BMadrid/Protector/Protector3.aspx
/esp/BMadrid/Protector/Protector4.aspx
#
/esp/aspx/Miembros/Miembros.aspx?tipo=T
javascript:sacaVentana('/docs/Miembros/Comosermiembro.pdf')
/esp/aspx/Regulacion/Regulacion.aspx
/docs/Miembros/TarifasRectoras.pdf
/esp/BMadrid/Localizacion.aspx
/esp/BMadrid/Contacto.aspx
#
/esp/aspx/Mercados/Precios.asp

In [27]:
for link in soup.find('div', {'id':'Base'}).find_all('a', string='Acciones'):
    if 'href' in link.attrs:
        tails =link.attrs['href']
        #print(tails)[:1]
        print(link.attrs['href']) 

/esp/aspx/Mercados/Precios.aspx?indice=ESI100000000
/esp/aspx/Mercados/Precios.aspx?indice=ESI100000000


In [35]:
tails = [link.attrs['href'] for link in soup.find('div', {'id':'Base'}).find_all(
    'a', string='Acciones') if 'href' in link.attrs]


/esp/aspx/Mercados/Precios.aspx?indice=ESI100000000


In [36]:
tail = tails[-1]
tail

'/esp/aspx/Mercados/Precios.aspx?indice=ESI100000000'

In [37]:
url = 'https://www.bolsamadrid.es'
newUrl = url+tails[-1]
newUrl

'https://www.bolsamadrid.es/esp/aspx/Mercados/Precios.aspx?indice=ESI100000000'

In [49]:
newpage = urlopen(newUrl)
soup = BeautifulSoup(newpage,"html.parser")


In [50]:
accions = []
acc_tables = soup.find_all('table', class_="TblPort")

for table in acc_tables:
    headers =[]
    rows = table.find_all('tr')
    #start by fetching the header cells from the first row to determine the field names
    for header in table.find('tr').find_all('th'):
        headers.append(header.text)
    #then go through all the rows except the first one
    for row in table.find_all('tr')[1:]:
        values = []
        #And get the column cells, the first one being inside a th-tag
        for col in row.find_all(['th', 'td']):
            values.append(col.text)
        if values:
            acc_dict = {headers[i]:values[i] for i in range(len(values))}
            accions.append(acc_dict)


In [52]:
accions[:2] 

[{'Nombre': 'IBEX 35®',
  'Anterior': '8.991,50',
  'Último': '8.988,20',
  '% Dif.': '-0,04',
  'Máximo': '9.020,70',
  'Mínimo': '8.972,10',
  'Fecha': '02/09/2021',
  'Hora': '13:00:23',
  '% Dif.Año 2021': '11,33'},
 {'Nombre': 'ACCIONA',
  'Últ.': '141,2000',
  '% Dif.': '1,00',
  'Máx.': '141,7000',
  'Mín.': '139,5000',
  'Volumen': '14.787',
  'Efectivo (miles €)': '2.083,61',
  'Fecha': '02/09/2021',
  'Hora': '12:59:09'}]

In [46]:
del accions[0]

In [47]:
import pandas as pd 

df = pd.DataFrame(accions)
df.head()

,Nombre,Últ.,% Dif.,Máx.,Mín.,Volumen,Efectivo (miles €),Fecha,Hora
0,ACCIONA,"141,1000","0,93","141,7000","139,5000",12.954,"1.824,78",02/09/2021,12:27:08
1,ACERINOX,"11,4550","-0,52","11,5750","11,4400",174.838,"2.012,16",02/09/2021,12:33:27
2,ACS,"23,5900","0,34","23,6400","23,4000",111.737,"2.631,38",02/09/2021,12:30:47
3,AENA,"137,8000","-0,22","139,2500","136,9500",11.588,"1.601,74",02/09/2021,12:33:00
4,ALMIRALL,"14,7300","0,68","14,7500","14,4500",115.102,"1.687,89",02/09/2021,12:31:06


### amb Selenium.

In [68]:
from selenium import webdriver
import pandas as pd 
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

url = 'https://www.bolsamadrid.es'

In [69]:
driver =webdriver.Chrome()
driver.get(url)
WebDriverWait(driver, 20).until(
    EC.presence_of_element_located((By.XPATH, '//*[@id="Base"]/table/tbody/tr[4]/td[2]/div[1]/div[2]/table/tbody/tr[1]/td[1]/a'))).click()

Així obtenim les dades de la pàgina, però les volem tenir en un DataFrame per poder treballar amb elles...

In [20]:
taula = driver.find_elements(By.XPATH,'//*[@id="ctl00_Contenido_tblAcciones"]')
for i in taula:
    print(i.text)


Nombre Últ. % Dif. Máx. Mín. Volumen Efectivo (miles €) Fecha Hora
ACCIONA 142,8000 0,42 143,6000 142,5000 6.590 942,69 03/09/2021 09:26:19
ACERINOX 11,4100 -0,57 11,5400 11,3900 39.807 455,16 03/09/2021 09:25:19
ACS 23,6100 0,64 23,6900 23,4300 24.233 571,91 03/09/2021 09:25:59
AENA 138,4500 -0,04 138,7000 138,0500 2.907 402,46 03/09/2021 09:25:47
ALMIRALL 14,7600 0,20 14,8000 14,6600 25.297 372,83 03/09/2021 09:19:20
AMADEUS 52,7000 -0,26 52,8200 52,3400 47.711 2.513,06 03/09/2021 09:25:05
ARCELORMIT. 28,3850 -0,98 28,6250 28,1750 48.224 1.367,62 03/09/2021 09:26:14
B.SANTANDER 3,1140 -0,81 3,1350 3,1075 1.188.639 3.709,87 03/09/2021 09:26:11
BA.SABADELL 0,6102 -0,39 0,6144 0,6096 468.106 286,14 03/09/2021 09:24:03
BANKINTER 4,9520 -0,74 4,9850 4,9500 30.491 151,53 03/09/2021 09:26:08
BBVA 5,6240 0,27 5,6400 5,5860 564.975 3.171,47 03/09/2021 09:26:28
CAIXABANK 2,6680 -0,15 2,6690 2,6560 379.216 1.008,69 03/09/2021 09:26:30
CELLNEX 58,0000 -0,68 58,5200 57,9600 40.593 2.364,43 03/09/

ara n'extreurem les dades, dels titols, i els convinarem en un DataFrame.

In [75]:
headers = [header.text for header in driver.find_elements(By.XPATH,'//*[@id="ctl00_Contenido_tblAcciones"]/tbody/tr/th')]
headers

['Nombre',
 'Últ.',
 '% Dif.',
 'Máx.',
 'Mín.',
 'Volumen',
 'Efectivo (miles €)',
 'Fecha',
 'Hora']

In [82]:
dades = [dada.text for dada in driver.find_elements(By.XPATH,'//*[@id="ctl00_Contenido_tblAcciones"]/tbody/tr/td')] 
dades

['ACCIONA',
 '142,9000',
 '0,49',
 '143,6000',
 '142,2000',
 '16.050',
 '2.292,52',
 '03/09/2021',
 '10:31:19',
 'ACERINOX',
 '11,4750',
 '0,00',
 '11,5400',
 '11,3900',
 '75.096',
 '858,85',
 '03/09/2021',
 '10:33:42',
 'ACS',
 '23,5900',
 '0,55',
 '23,6900',
 '23,4300',
 '60.966',
 '1.437,60',
 '03/09/2021',
 '10:31:30',
 'AENA',
 '137,5500',
 '-0,69',
 '138,7000',
 '137,4000',
 '5.129',
 '709,21',
 '03/09/2021',
 '10:31:39',
 'ALMIRALL',
 '14,7200',
 '-0,07',
 '14,8000',
 '14,6600',
 '55.893',
 '823,67',
 '03/09/2021',
 '10:27:37',
 'AMADEUS',
 '52,1800',
 '-1,25',
 '52,8200',
 '52,1400',
 '85.166',
 '4.476,87',
 '03/09/2021',
 '10:33:51',
 'ARCELORMIT.',
 '28,5500',
 '-0,40',
 '28,6250',
 '28,1750',
 '71.983',
 '2.043,79',
 '03/09/2021',
 '10:33:41',
 'B.SANTANDER',
 '3,1190',
 '-0,65',
 '3,1350',
 '3,1060',
 '2.544.072',
 '7.930,87',
 '03/09/2021',
 '10:33:53',
 'BA.SABADELL',
 '0,6114',
 '-0,20',
 '0,6144',
 '0,6070',
 '1.458.313',
 '889,48',
 '03/09/2021',
 '10:33:33',
 'BANKINT

In [89]:
#dividirem les dades en llistes q continguin la informació de cada companyia
#mida dels grups, ha de ser ingual de llarg q el grup del titols
n = len(headers)
  
# ho farem amb una comprehension list
dades_final = [dades[i * n:(i + 1) * n] for i in range((len(dades) + n - 1) // n )] 
 

In [88]:
import pandas as pd 

df = pd.DataFrame(dades_final, columns=headers)
df.head()



,Nombre,Últ.,% Dif.,Máx.,Mín.,Volumen,Efectivo (miles €),Fecha,Hora
0,ACCIONA,"142,9000","0,49","143,6000","142,2000",16.050,"2.292,52",03/09/2021,10:31:19
1,ACERINOX,"11,4750","0,00","11,5400","11,3900",75.096,"858,85",03/09/2021,10:33:42
2,ACS,"23,5900","0,55","23,6900","23,4300",60.966,"1.437,60",03/09/2021,10:31:30
3,AENA,"137,5500","-0,69","138,7000","137,4000",5.129,"709,21",03/09/2021,10:31:39
4,ALMIRALL,"14,7200","-0,07","14,8000","14,6600",55.893,"823,67",03/09/2021,10:27:37


Ja ho tenim llest per treballar amb les dades, exportar-ho a un document, etc.

In [ ]:
driver.quit()

### Amb selenium fent un filtre previ

In [90]:
from selenium import webdriver
import pandas as pd 
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

url = 'https://www.bolsamadrid.es'

In [91]:
driver =webdriver.Chrome()
driver.get(url)
WebDriverWait(driver, 20).until(
    EC.presence_of_element_located((By.XPATH, '//*[@id="Base"]/table/tbody/tr[4]/td[2]/div[1]/div[2]/table/tbody/tr[1]/td[1]/a'))).click()

Si vulguessim fer servir el filtre d'inici de la pàgina, afegririem el següent:

In [95]:
from selenium.webdriver.support.ui import Select
select = Select(driver.find_element_by_xpath('//*[@id="SelSector"]'));
select.select_by_visible_text("Tecnología y Telecomunicaciones");
driver.find_element_by_xpath('//*[@id="ctl00_Contenido_Consultar"]').click() 
WebDriverWait(driver, 20)

<selenium.webdriver.support.wait.WebDriverWait (session="ea19be5ef6c8b4e5570bf07f9de2f3f2")>

In [96]:
taula = driver.find_elements(By.XPATH,'//*[@id="ctl00_Contenido_tblAcciones"]')
for i in taula:
    print(i.text)


Nombre Mercado Últ. % Dif. Máx. Mín. Volumen Efectivo (miles €) Fecha Hora
AETERNAL M. MAB - EE 8,5500 4,91 8,5500 8,5500 810 6,93 03/09/2021 12:00:11
AGILE MAB - EE 7,4000 -1,33 7,4000 7,4000 300 2,22 03/09/2021 10:10:52
ALTIA MAB - EE 27,8000 - - - - - 02/09/2021 Cierre
AMADEUS M.Continuo 51,9400 -1,70 52,8200 51,7400 162.268 8.483,94 03/09/2021 12:21:30
AMERICA M. L Latibex 0,8000 - - - - - 02/09/2021 Cierre
AMPER M.Continuo 0,1850 -0,54 0,1868 0,1846 189.064 34,99 03/09/2021 12:10:27
CATENON MAB - EE 1,1000 2,80 1,1000 1,0700 14.190 15,51 03/09/2021 10:48:43
CELLNEX M.Continuo 57,9800 -0,72 58,5200 57,7400 137.508 7.987,96 03/09/2021 12:21:18
CLEVER MAB - EE 0,2340 - - - - - 02/09/2021 Cierre
CUATROOCHENT MAB - EE 17,0000 - - - - - 02/09/2021 Cierre
DOMINION M.Continuo 4,5150 0,33 4,5500 4,4950 65.767 297,26 03/09/2021 12:08:43
EURONA MAB - EE 0,1960 - - - - - Suspendido 30/04/2020
EZENTIS M.Continuo 0,3550 -0,70 0,3575 0,3540 93.198 33,08 03/09/2021 12:14:37
FACEPHI MAB - EE 5,100

ara n'extreurem les dades, dels titols, i els convinarem en un DataFrame.

In [97]:
headers = [header.text for header in driver.find_elements(By.XPATH,'//*[@id="ctl00_Contenido_tblAcciones"]/tbody/tr/th')]
headers

['Nombre',
 'Mercado',
 'Últ.',
 '% Dif.',
 'Máx.',
 'Mín.',
 'Volumen',
 'Efectivo (miles €)',
 'Fecha',
 'Hora']

In [98]:
dades = [dada.text for dada in driver.find_elements(By.XPATH,'//*[@id="ctl00_Contenido_tblAcciones"]/tbody/tr/td')] 
dades

['AETERNAL M.',
 'MAB - EE',
 '8,5500',
 '4,91',
 '8,5500',
 '8,5500',
 '810',
 '6,93',
 '03/09/2021',
 '12:00:11',
 'AGILE',
 'MAB - EE',
 '7,4000',
 '-1,33',
 '7,4000',
 '7,4000',
 '300',
 '2,22',
 '03/09/2021',
 '10:10:52',
 'ALTIA',
 'MAB - EE',
 '27,8000',
 '-',
 '-',
 '-',
 '-',
 '-',
 '02/09/2021',
 'Cierre',
 'AMADEUS',
 'M.Continuo',
 '51,9400',
 '-1,70',
 '52,8200',
 '51,7400',
 '162.268',
 '8.483,94',
 '03/09/2021',
 '12:21:30',
 'AMERICA M. L',
 'Latibex',
 '0,8000',
 '-',
 '-',
 '-',
 '-',
 '-',
 '02/09/2021',
 'Cierre',
 'AMPER',
 'M.Continuo',
 '0,1850',
 '-0,54',
 '0,1868',
 '0,1846',
 '189.064',
 '34,99',
 '03/09/2021',
 '12:10:27',
 'CATENON',
 'MAB - EE',
 '1,1000',
 '2,80',
 '1,1000',
 '1,0700',
 '14.190',
 '15,51',
 '03/09/2021',
 '10:48:43',
 'CELLNEX',
 'M.Continuo',
 '57,9800',
 '-0,72',
 '58,5200',
 '57,7400',
 '137.508',
 '7.987,96',
 '03/09/2021',
 '12:21:18',
 'CLEVER',
 'MAB - EE',
 '0,2340',
 '-',
 '-',
 '-',
 '-',
 '-',
 '02/09/2021',
 'Cierre',
 'CUATROO

In [99]:
#dividirem les dades en llistes q continguin la informació de cada companyia
#mida dels grups, ha de ser ingual de llarg q el grup del titols
n = len(headers)
  
# ho farem amb una comprehension list
dades_final = [dades[i * n:(i + 1) * n] for i in range((len(dades) + n - 1) // n )] 
 

In [100]:
import pandas as pd 

df = pd.DataFrame(dades_final, columns=headers)
df.head()



,Nombre,Mercado,Últ.,% Dif.,Máx.,Mín.,Volumen,Efectivo (miles €),Fecha,Hora
0,AETERNAL M.,MAB - EE,"8,5500","4,91","8,5500","8,5500",810,"6,93",03/09/2021,12:00:11
1,AGILE,MAB - EE,"7,4000","-1,33","7,4000","7,4000",300,"2,22",03/09/2021,10:10:52
2,ALTIA,MAB - EE,"27,8000",-,-,-,-,-,02/09/2021,Cierre
3,AMADEUS,M.Continuo,"51,9400","-1,70","52,8200","51,7400",162.268,"8.483,94",03/09/2021,12:21:30
4,AMERICA M. L,Latibex,"0,8000",-,-,-,-,-,02/09/2021,Cierre


In [101]:
driver.quit()

## Web scraping la Borsa de Madrid-Documentació

arxiu txt que podeu trobar en aquest link [Dataset Documentation](http://localhost:8888/edit/Documents/IT%20academy/DataScience/documentsTreball/S16%20ML%20avancat/S06_T02%20Dataset%20Documentation)



## Scrapy